In [3]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv1D, MaxPooling1D, UpSampling1D, Flatten, Reshape, BatchNormalization, Dropout
from tensorflow.keras.models import Model
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error, median_absolute_error
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Load the dataset
file_path = "train_FD004_processed.csv"
df = pd.read_csv(file_path)

# Cap RUL at a maximum value (piecewise linear RUL, common in CMAPSS datasets)
max_rul = 130  # Typical cap for CMAPSS datasets
df['RUL'] = df['RUL'].clip(upper=max_rul)

# Define feature and target columns
WINDOW_SIZE = 30
FEATURE_COLS = [col for col in df.columns if col.startswith('op_setting_') or col.startswith('sensor_measurement_')]
TARGET_COL = 'RUL'

# Feature selection: Remove features with near-zero variance
variances = df[FEATURE_COLS].var()
selected_features = variances[variances > 0.01].index.tolist()  # Keep features with variance > 0.01
print(f"Selected features: {selected_features}")
df = df[['unit_number'] + selected_features + [TARGET_COL]]

# Normalize the features
scaler_features = StandardScaler()
df[selected_features] = scaler_features.fit_transform(df[selected_features])

# Function to create sequences
def create_sequences(data, window_size, feature_cols):
    X = []
    for unit in data['unit_number'].unique():
        unit_data = data[data['unit_number'] == unit]
        feature_data = unit_data[feature_cols].values
        for i in range(len(unit_data) - window_size):
            X.append(feature_data[i:i+window_size])
    return np.array(X)

# Create sequences
X = create_sequences(df, WINDOW_SIZE, selected_features)

# Split the data
X_train, X_val = train_test_split(X, test_size=0.2, random_state=42)

# Build the Enhanced Autoencoder
input_shape = X.shape[1:]  # (window_size, num_features)
latent_dim = 64  # Increased for better information retention

# Encoder
inputs = Input(shape=input_shape)
x = Conv1D(filters=32, kernel_size=3, activation='relu', padding='same')(inputs)
x = BatchNormalization()(x)
x = MaxPooling1D(pool_size=2, padding='same')(x)  # Reduces from 30 to 15
x = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling1D(pool_size=2, padding='same')(x)  # Reduces from 15 to 8
x = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(x)  # Added layer
x = BatchNormalization()(x)
x = Flatten()(x)
encoded = Dense(latent_dim, activation='relu', name='latent')(x)

# Decoder
x = Dense(8 * input_shape[-1])(encoded)  # 8 = temporal steps after second pooling, input_shape[-1] = num_features
x = Reshape((8, input_shape[-1]))(x)  # Reshape to (8, num_features)
x = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = UpSampling1D(size=2)(x)  # Upsamples from 8 to 16
x = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = UpSampling1D(size=2)(x)  # Upsamples from 16 to 32
x = Conv1D(filters=32, kernel_size=3, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Cropping1D(cropping=(1, 1))(x)  # Crop 1 from start and 1 from end (32 -> 30)
decoded = Conv1D(filters=input_shape[-1], kernel_size=3, activation='linear', padding='same')(x)  # Output (30, num_features)

# Autoencoder model
autoencoder = Model(inputs, decoded)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
autoencoder.compile(optimizer=optimizer, loss='mse')
autoencoder.summary()

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Train the autoencoder
history = autoencoder.fit(
    X_train, X_train,
    validation_data=(X_val, X_val),
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping, lr_scheduler],
    verbose=1
)

# Verify reconstruction and calculate all test parameters
decoded_val = autoencoder.predict(X_val)
print("Input shape:", X_val.shape)
print("Decoded shape:", decoded_val.shape)
print("Input sample (first time step):", X_val[0, 0, :])
print("Decoded sample (first time step):", decoded_val[0, 0, :])

# Flatten the data for metric calculation (across all time steps and features)
X_val_flat = X_val.reshape(X_val.shape[0], -1)  # (samples, 30*num_features)
decoded_val_flat = decoded_val.reshape(decoded_val.shape[0], -1)  # (samples, 30*num_features)

# Calculate all test parameters
mse = mean_squared_error(X_val_flat, decoded_val_flat)
mae = mean_absolute_error(X_val_flat, decoded_val_flat)
rmse = np.sqrt(mse)
r2 = r2_score(X_val_flat, decoded_val_flat)
mape = mean_absolute_percentage_error(X_val_flat, decoded_val_flat)
medae = median_absolute_error(X_val_flat, decoded_val_flat)

# Print all test parameters
print(f"MSE Autoencoder: {mse:.4f}")
print(f"MAE Autoencoder: {mae:.4f}")
print(f"RMSE Autoencoder: {rmse:.4f}")
print(f"R² Score Autoencoder: {r2:.4f}")
print(f"MAPE Autoencoder: {mape:.2f}%")
print(f"Median Absolute Error Autoencoder: {medae:.4f}")

# Save the autoencoder
autoencoder.save("enhanced_autoencoder_rul_model.keras")

Selected features: ['op_setting_1', 'op_setting_2', 'op_setting_3', 'sensor_measurement_1', 'sensor_measurement_2', 'sensor_measurement_3', 'sensor_measurement_4', 'sensor_measurement_5', 'sensor_measurement_6', 'sensor_measurement_7', 'sensor_measurement_8', 'sensor_measurement_9', 'sensor_measurement_10', 'sensor_measurement_11', 'sensor_measurement_12', 'sensor_measurement_13', 'sensor_measurement_14', 'sensor_measurement_15', 'sensor_measurement_16', 'sensor_measurement_17', 'sensor_measurement_18', 'sensor_measurement_19', 'sensor_measurement_20', 'sensor_measurement_21']


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 30, 24)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_6 (Conv1D)               │ (None, 30, 32)         │         2,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 30, 32)         │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 15, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_7 (Conv1D)               │ (None, 15, 64)         │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 15, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 8, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_8 (Conv1D)               │ (None, 8, 128)         │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 8, 128)         │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ latent (Dense)                  │ (None, 64)             │        65,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 192)            │        12,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_1 (Reshape)             │ (None, 8, 24)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_9 (Conv1D)               │ (None, 8, 128)         │         9,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 8, 128)         │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling1d_2 (UpSampling1D)  │ (None, 16, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_10 (Conv1D)              │ (None, 16, 64)         │        24,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 16, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling1d_3 (UpSampling1D)  │ (None, 32, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_11 (Conv1D)              │ (None, 32, 32)         │         6,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 32, 32)         │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cropping1d_1 (Cropping1D)       │ (None, 30, 32)         │             

 Total params: 155,608 (607.84 KB)

 Trainable params: 154,712 (604.34 KB)

 Non-trainable params: 896 (3.50 KB)

Epoch 1/50
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 20s 9ms/step - loss: 0.6571 - val_loss: 0.1070 - learning_rate: 0.0010
Epoch 2/50
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.0594 - val_loss: 0.0544 - learning_rate: 0.0010
Epoch 3/50
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - loss: 0.0492 - val_loss: 0.0463 - learning_rate: 0.0010
Epoch 4/50
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - loss: 0.0443 - val_loss: 0.0453 - learning_rate: 0.0010
Epoch 5/50
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 0.0420 - val_loss: 0.0387 - learning_rate: 0.0010
Epoch 6/50
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - loss: 0.0382 - val_loss: 0.0341 - learning_rate: 0.0010
Epoch 7/50
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 0.0325 - val_loss: 0.0334 - learning_rate: 0.0010
Epoch 8/50
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - loss: 0.0279 - val_loss: 0.0250 - learning_rate: 0.0010
Epoch 9/50
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.0258 - val_loss: 0.0220 - learning_r